#####  环境2：阿里云 ubuntu  python27 tensorflowf1.4.0

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'1.4.0'

#####  环境2：小米Air  Win10  python3 tensorflowf1.12.0

In [2]:
tf.__version__

'1.12.0'

## 6.1 图像识别问题简介及经典数据集

#### CIFAR数据集简介：
（1）10个不同种类60000张图片
（2）像素是32*32

####  ImageNet是一个基于WordNet的大型图像数据库。在其中，近1500万张图片关联到WordNet的20000个名词同义词集上。

## 6.2 卷积神经网络简介 

#### 对与卷积神经网络，相邻两层之间只有部分节点相连，为了展示每一层神经元的维度，一般会将每一层卷积层的节点组织成一个三维矩阵

#### 卷积神经网络的结构组成：
1. 输入层
2. 卷积层：通过卷积层处理过的节点矩阵会变深；
3. 池化层：池化层不会改变三维矩阵的深度但是会缩小矩阵的大小。
4. 全连接层：可以将卷积层和池化层看成自动图像特征提取的过程。在特征提取完之后仍然需要使用全连接层完成分类任务。
5. Softmax层;主要用于分类问题。

## 6.3 卷积神经网络常用的结构

### 6.3.1 卷积层

- 过滤器（内核）：可以将当前神经网络上的一个子结点矩阵转化为下一层神经网络上一个单位节点矩阵（长宽为1，深度不限）。
- 过滤器的长和宽是人工指定的。常用过滤器尺寸有3×3、5×5。

#### 如果要避免尺寸变化，可以在当前层矩阵边界填充0

![1755](../TfInActionImg/1755.PNG)

- 在同时使用全0填充时结果矩阵的大小
![1800](../TfInActionImg/1800.PNG)
- 不使用0填充的结果矩阵大小
![1801](../TfInActionImg/1801.PNG)

#### 在卷积神经网络中，每一个卷积层中使用的过滤器参数都是一样的，这样使得图像上的内容不受位置影响

In [ ]:
# no-exe code
# Tensorflow实现卷积层样例：

filter_weight = tf.get_variable(
    'weights',[5,5,3,6],#过滤器尺寸5*5，深度3，当前层节点矩阵深度16
    initializer = tf.truncated_normal_initializer(stddev = 0.1)
)
biases = tf.get_variable(
    'biases',[16], #过滤器的深度，同时也是下一层节点的矩阵深度
    initializer = tf.constant_initializer(0.1)
)
conv = tf.nn.conv2d(
    input,#四维数，后三维表示当前层节点矩阵，比如input[2,:,:,:]表示第三张图片
    filter_weight,# 卷积层权重
    strides = [1,1,1,1],# 不同维度上的步长但是第一个和最后一个数必须是1
    padding = 'SAME'# SAME表示添加全0填充，VALID表示不添加bias
)
bias = tf.nn.bias_add(conv,biases) # ???
actived_conv = tf.nn.relu(bias)

### 6.3.2 池化层

#### 池化层作用：
- 缩小矩阵尺寸，从而减少全连接层中的参数；
- 使用池化层既可以加快计算速度也有防止过拟合问题的作用；

#### 常用的池化层： 最大池化层、平均池化层

In [ ]:
# no-exe code
# Tensorflow实现池化层样例：
pool = tf.nn.max_pool(
    actived_conv,
    ksize = [1,3,3,1],
    strides = [1,2,2,1],
    padding = 'SAME'
)

## 6.4 经典卷积网络模型

### 6.4.1 LeNet-5 模型

#### 每层参数数目分析：
![2030](../TfInActionImg/2030.PNG)
![2031](../TfInActionImg/2031.PNG)

#### Tensorflow实现CNN示例：

In [1]:
# LeNet5_train
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import LeNet5_infernece
import os
import numpy as np

# 1. 定义神经网络相关的参数
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.01
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 800
MOVING_AVERAGE_DECAY = 0.99

# 2.定义训练过程
def train(mnist):
    # 定义输出为4维矩阵的placeholder
    x = tf.placeholder(tf.float32, [
            BATCH_SIZE,
            LeNet5_infernece.IMAGE_SIZE,
            LeNet5_infernece.IMAGE_SIZE,
            LeNet5_infernece.NUM_CHANNELS],
        name='x-input')
    y_ = tf.placeholder(tf.float32, [None, LeNet5_infernece.OUTPUT_NODE], name='y-input')
    
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y = LeNet5_infernece.inference(x,False,regularizer)
    global_step = tf.Variable(0, trainable=False)

    # 定义损失函数、学习率、滑动平均操作以及训练过程。
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY,
        staircase=True)

    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')
        
    # 初始化TensorFlow持久化类。
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)

            reshaped_xs = np.reshape(xs, (
                BATCH_SIZE,
                LeNet5_infernece.IMAGE_SIZE,
                LeNet5_infernece.IMAGE_SIZE,
                LeNet5_infernece.NUM_CHANNELS))
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: reshaped_xs, y_: ys})

            if i % 100 == 0:
                print("After %d training step(s), loss on training batch is %g." % (step, loss_value))
                
# 3.主程序入口
def main(argv=None):
    mnist = input_data.read_data_sets("../data/mnist/", one_hot=True)
    
    train(mnist)

if __name__ == '__main__':
    tf.reset_default_graph() 
    main()

C:\softwareYXQ\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
After 1 training step(s), loss on training batch is 4.66097.
After 101 training step(s), loss on training batch is 1.12016.
After 201 training step(s), loss on training batch is 0.928684.
After 301 training step(s), loss on training batch is 0.799152.
After 401 training step(s), loss on t

#### 卷积神经网络架构： 输入层->（卷积层+池化层?）+->全连接层+

### 6.4.2 Inception模型

![2117](../TfInActionImg/2117.PNG)

#### 代码见P158 

## 6.5 卷积神经网络迁移学习

### 6.5.1 迁移学习介绍

#### 数据集：http://download.tensorflow.org/example_images/flower_photos.tgz
- 注：该数据集需要通过科学上网下载，该数据集未同步到github

#### 代码见：
- data_process 
- fine_tuning